In [347]:
import pandas as pd

In [348]:
transaction = pd.read_csv('transactions.csv')
account = pd.read_csv('account_info.csv')
cities_to_states = pd.read_csv('uscities.csv')

In [349]:
display(transaction)

,account_number,transaction_datetime,transaction_amount,post_date,merchant_number,merchant_description,merchant_category_code,transaction_number
0,26522,12182020 15:29:12,1.06-,12192020,000000000326552,MAVERIK #488 DRAPER LAUS,5541,3
1,26522,12302020 08:22:36,1.72-,12312020,900185000000000,HARMONS - BANGERTER DRAPER LA,5411,8
2,11111,02192021 09:55:25,1009.90-,2202021,527021002118976,CFA Institute 4349515499 TX,8299,1
3,11111,02092021 11:15:52,14.00-,2112021,000225124800999,CHAPULTEPEC LUPITA HOUSTON TX,5812,2
4,26522,01272021 09:34:44,0.90-,1272021,650000010604109,PCBC SPRINGVILLE SPRINGVILLE LA,5814,2
...,...,...,...,...,...,...,...,...
2517,37222,02272021 15:27:57,4.85-,3012021,000000002355702,TIGER EXPRESS Los Angeles CA,5499,2
2518,37222,04012021 06:17:18,50.04-,4022021,053894000053360,SHELL OIL Los Angeles CA,5814,3
2519,37222,03122021 07:33:49,45.73-,3132021,4445025811745,SHELL OIL\t\tLos Angeles CA,5812,4
2520,37222,01212021 12:28:59,125.00-,1232021,000021668538000,SHELL OIL Los Angeles CA,5812,5


In [350]:
cities_to_states.drop(cities_to_states.columns[range(4,17)],axis=1,inplace=True)
cities_to_states['city'] = cities_to_states['city'].map(lambda x: x.lower())
cities_to_states['city_ascii'] = cities_to_states['city_ascii'].map(lambda x: x.lower())
display(cities_to_states)

,city,city_ascii,state_id,state_name
0,new york,new york,NY,New York
1,los angeles,los angeles,CA,California
2,chicago,chicago,IL,Illinois
3,miami,miami,FL,Florida
4,dallas,dallas,TX,Texas
...,...,...,...,...
30404,drummond,drummond,ID,Idaho
30405,lost springs,lost springs,WY,Wyoming
30406,provo,provo,SD,South Dakota
30407,goldcreek,goldcreek,MT,Montana


In [351]:
# uscities.csv contains a list of cities and states in the US. 
# We will use this to determine the merchant name and state of the transaction.
# when we see a word in merchant_description that includes in the cities_to_states 'city' or 'city_ascii' column, we think that is the end of the merchant name.
# so all the words before that are the merchant name.
#  and the last word is the state.

# we will use this function to get the merchant name and state from the merchant_description column
# eg. 'WAL-MART STORES #  0001  Los Angeles TX' -> 'WAL-MART STORES #  0001', 'TX'
# eg. 'WAL-MART STORES #  0001  New York NY' -> 'WAL-MART STORES #  0001', 'NY'

def get_merchant_name(merchant_description):

    # find all the substrings of merchant_description
    #  and then delete the substrings that are in cities_to_states['city'] or cities_to_states['city_ascii']
    #  and the state also corresponds to the state of the city in cities_to_states
    #  the remaining string are the merchant name, and the last word is the state
    
    state = merchant_description.split()[-1][:2]
    new_merchant_description = merchant_description.replace(state,'').lower().split()
    merchant_description = merchant_description.split()
    sub_strings = []
    for i in range(len(new_merchant_description)):
        for j in range(i+1,len(new_merchant_description)+1):
            sub_strings.append(' '.join(new_merchant_description[i:j]))

    find_at_least_once = False
    for sub_string in sub_strings:
        if sub_string in cities_to_states['city'].values or sub_string in cities_to_states['city_ascii'].values:
            if cities_to_states[cities_to_states['city']==sub_string]['state_id'].values[0] == state or cities_to_states[cities_to_states['city_ascii']==sub_string]['state_id'].values[0] == state:
                # find the index of the sub_string in merchant_description
                if sub_string == 'young':
                    continue
                else:
                    find_at_least_once = True
                    loc = new_merchant_description.index(sub_string.split()[0])
                    # delete the sub_string and the words after it
                    merchant_description = merchant_description[:loc]
                
    if find_at_least_once == False:
        
        merchant_name = ' '.join(merchant_description[:-1])
                
    else:
        merchant_name = ' '.join(merchant_description)
    
    
    
    return merchant_name
 

In [352]:
# split merchant_description into merchant_name and merchant_state
# keep last word as merchant_state, and the rest as merchant_name

transaction['merchant_name'] = transaction['merchant_description'].map (lambda x: get_merchant_name(x))
transaction['merchant_state'] = transaction['merchant_description'].map (lambda x: x.split()[-1][:2])

In [353]:
display(transaction)

,account_number,transaction_datetime,transaction_amount,post_date,merchant_number,merchant_description,merchant_category_code,transaction_number,merchant_name,merchant_state
0,26522,12182020 15:29:12,1.06-,12192020,000000000326552,MAVERIK #488 DRAPER LAUS,5541,3,MAVERIK #488 DRAPER,LA
1,26522,12302020 08:22:36,1.72-,12312020,900185000000000,HARMONS - BANGERTER DRAPER LA,5411,8,HARMONS - BANGERTER DRAPER,LA
2,11111,02192021 09:55:25,1009.90-,2202021,527021002118976,CFA Institute 4349515499 TX,8299,1,CFA Institute 4349515499,TX
3,11111,02092021 11:15:52,14.00-,2112021,000225124800999,CHAPULTEPEC LUPITA HOUSTON TX,5812,2,CHAPULTEPEC LUPITA,TX
4,26522,01272021 09:34:44,0.90-,1272021,650000010604109,PCBC SPRINGVILLE SPRINGVILLE LA,5814,2,PCBC SPRINGVILLE SPRINGVILLE,LA
...,...,...,...,...,...,...,...,...,...,...
2517,37222,02272021 15:27:57,4.85-,3012021,000000002355702,TIGER EXPRESS Los Angeles CA,5499,2,TIGER EXPRESS,CA
2518,37222,04012021 06:17:18,50.04-,4022021,053894000053360,SHELL OIL Los Angeles CA,5814,3,SHELL OIL,CA
2519,37222,03122021 07:33:49,45.73-,3132021,4445025811745,SHELL OIL\t\tLos Angeles CA,5812,4,SHELL OIL,CA
2520,37222,01212021 12:28:59,125.00-,1232021,000021668538000,SHELL OIL Los Angeles CA,5812,5,SHELL OIL,CA


In [354]:
# map merchant_state to full state name
states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

states_lists = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

In [355]:
# delete merchant_description

del(transaction['merchant_description'])

In [356]:
# mapping abbrevations with will state name
transaction['merchant_state'] = transaction['merchant_state'].map(lambda x: states_lists[x] if x in states else x)

account[' state'] = account[' state'].map(lambda x: states_lists[x] if x in states else x)


In [357]:
transaction

,account_number,transaction_datetime,transaction_amount,post_date,merchant_number,merchant_category_code,transaction_number,merchant_name,merchant_state
0,26522,12182020 15:29:12,1.06-,12192020,000000000326552,5541,3,MAVERIK #488 DRAPER,Louisiana
1,26522,12302020 08:22:36,1.72-,12312020,900185000000000,5411,8,HARMONS - BANGERTER DRAPER,Louisiana
2,11111,02192021 09:55:25,1009.90-,2202021,527021002118976,8299,1,CFA Institute 4349515499,Texas
3,11111,02092021 11:15:52,14.00-,2112021,000225124800999,5812,2,CHAPULTEPEC LUPITA,Texas
4,26522,01272021 09:34:44,0.90-,1272021,650000010604109,5814,2,PCBC SPRINGVILLE SPRINGVILLE,Louisiana
...,...,...,...,...,...,...,...,...,...
2517,37222,02272021 15:27:57,4.85-,3012021,000000002355702,5499,2,TIGER EXPRESS,California
2518,37222,04012021 06:17:18,50.04-,4022021,053894000053360,5814,3,SHELL OIL,California
2519,37222,03122021 07:33:49,45.73-,3132021,4445025811745,5812,4,SHELL OIL,California
2520,37222,01212021 12:28:59,125.00-,1232021,000021668538000,5812,5,SHELL OIL,California


In [358]:
account

,last_name,first_name,street_address,unit,city,state,zip,dob,ssn,email_address,mobile_number,account_number
0,Dark,Joseph,7 B Blue Ridge Blvd,NaN,Brighton,Michigan,48116,3/1/1975,205-81-3991,jdarky@darakjy.org,5046218927,43991
1,James,Butt,6649 N Blue Gum St,NaN,New Orleans,Louisiana,70116,7/29/1965,508-60-8859,jbutt@gmail.com,7134926037,26522
2,Josephine,Darakjy,4 B Blue Ridge Blvd,NaN,Brighton,Michigan,48116,3/1/1975,205-81-3006,josephine_darakjy@darakjy.org,5046218927,31938
3,Art,Venere,8 W Cerritos Ave,#54,Bridgeport,New Jersey,8014,11/10/1947,793-96-7688,art@venere.org,8102929388,31094
4,Lenna,Paprocki,639 Main St,NaN,Anchorage,Alaska,99501,2/11/1986,581-28-7051,lpaprocki@hotmail.com,8566368749,18393
...,...,...,...,...,...,...,...,...,...,...,...,...
199,Claribel,Varriano,6 Harry L Dr,#6327,Perrysburg,Ohio,43551,11/24/1958,337-41-6133,claribel_varriano@cox.net,9739433423,23312
200,Benton,Skursky,47939 Porter Ave,NaN,Gardena,California,90248,6/26/1975,289-76-4017,benton.skursky@aol.com,4195444900,35903
201,Hillary,Skulski,9 Wales Rd Ne,#914,Homosassa,Florida,34448,12/4/1984,418-85-4593,hillary.skulski@aol.com,3105792907,29162
202,Merilyn,Bayless,195 13n N,NaN,Santa Clara,California,95054,4/26/1943,919-99-7195,merilyn_bayless@cox.net,3522422570,16090


In [359]:
transaction.to_csv('transaction_cleaned.csv', index=False)  

In [360]:
account.to_csv('account_cleaned.csv', index=False)  

In [361]:
transaction2 = pd.read_csv('transaction_cleaned.csv')


In [362]:
transaction2['transaction_amount'] = transaction2['transaction_amount'].map(lambda x: x.replace('-',''))
transaction2['transaction_amount'] = transaction2['transaction_amount'].map(lambda x: x.replace('+',''))

In [363]:
transaction2

,account_number,transaction_datetime,transaction_amount,post_date,merchant_number,merchant_category_code,transaction_number,merchant_name,merchant_state
0,26522,12182020 15:29:12,1.06,12192020,000000000326552,5541,3,MAVERIK #488 DRAPER,Louisiana
1,26522,12302020 08:22:36,1.72,12312020,900185000000000,5411,8,HARMONS - BANGERTER DRAPER,Louisiana
2,11111,02192021 09:55:25,1009.90,2202021,527021002118976,8299,1,CFA Institute 4349515499,Texas
3,11111,02092021 11:15:52,14.00,2112021,000225124800999,5812,2,CHAPULTEPEC LUPITA,Texas
4,26522,01272021 09:34:44,0.90,1272021,650000010604109,5814,2,PCBC SPRINGVILLE SPRINGVILLE,Louisiana
...,...,...,...,...,...,...,...,...,...
2517,37222,02272021 15:27:57,4.85,3012021,000000002355702,5499,2,TIGER EXPRESS,California
2518,37222,04012021 06:17:18,50.04,4022021,053894000053360,5814,3,SHELL OIL,California
2519,37222,03122021 07:33:49,45.73,3132021,4445025811745,5812,4,SHELL OIL,California
2520,37222,01212021 12:28:59,125.00,1232021,000021668538000,5812,5,SHELL OIL,California


In [364]:
transaction2.to_csv('transaction_cleaned.csv', index=False)  